<a href="https://www.kaggle.com/code/testanother/lasso-regressor-with-parameter-tuning-369-score?scriptVersionId=129545571" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.compose import make_column_transformer
from xgboost import plot_importance
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from xgboost import plot_importance
import matplotlib.pyplot as plt
import shap
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
import eli5

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e14/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e14/test.csv')
df_submit = pd.read_csv('/kaggle/input/playground-series-s3e14/sample_submission.csv')

In [3]:
def pipeline_with_lasso(numeric_features, alpha=0.001, max_iter=10000):
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features)
        ]
    )

    model = linear_model.Lasso(alpha=alpha, max_iter=max_iter)

    clf = Pipeline([
         ('preprocessor', preprocessor),
         ('clf', model)
    ])
    
    return clf, model

def find_best_parameters(X_train, y_train):
    regressor = linear_model.Lasso(max_iter=10000)
    param_grid = {"alpha": [1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100] }
    search = GridSearchCV(regressor, param_grid, scoring='neg_mean_squared_error', cv=5).fit(X_train, y_train)
    return search.best_params_

def evaluate(y_test, y_pred):
    print("MAE", mean_absolute_error(y_test, y_pred))
    print("MSE", mean_squared_error(y_test,y_pred))
    print("MSLE", np.sqrt( mean_squared_log_error(y_test,y_pred) ))
    
def create_submission(file_name, y_sub, ids):
    output = pd.DataFrame()
    output['id'] = ids
    output['yield'] = y_sub
    output.to_csv(file_name, index=False)

In [4]:
X, y = df_train.drop(columns=['id', 'yield', 'AverageOfUpperTRange']), df_train['yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
#find_best_parameters(X_train, y_train)
# output -> {'alpha': 0.01}

In [6]:
pipeline, model = pipeline_with_lasso(X_train.columns)
pipeline.fit(X_train, y_train)

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.706e+09, tolerance: 1.828e+06


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'MaxOfLowerTRange',
       'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays',
       'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds'],
      dtype='object'))])),
                ('clf', Lasso(alpha=0.001, max_iter=10000))])

In [7]:
preds = pipeline.predict(X_test)

In [8]:
evaluate(y_test, preds)

MAE 377.4052314614496
MSE 347311.153235648
MSLE 0.1099050991443209


In [9]:
eli5.show_weights(model, top=-1, feature_names = X_train.columns.tolist())

In [10]:
y_preds = pipeline.predict(df_test.drop(columns=['id', 'AverageOfUpperTRange']))
#create_submission('new_sub_lasso_better.csv', y_preds, df_submit['id'])